In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

e:\cv-generation\train_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("dpo_requests.csv")
df.drop(columns="Unnamed: 0", axis=1, inplace=True)

In [3]:
checkpoint = "../Train/models/full-sft/full-sft"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to("cuda")

In [5]:
def predict(cv: str, temp: float, is_temp: bool=False):
    input_ids = tokenizer(
        [cv],
        max_length=600,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"].to("cuda")

    if is_temp:
        output_ids = model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4,
            temperature=temp,
            do_sample=True
        )[0]
    else:
        output_ids = model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)

    return summary

In [6]:
df["source"][3]

'Пол: женщина. Возраст: 14 лет. Желаемая зарплата: от 1500.0 рублей. Желаемая должность: Промоутер. Место проживания: Москва. Водительское удостоверение: отсутствует. Умения/навыки: раздача листовок, расклейка листовок. Опыт работы: без опыта работы'

In [7]:
predict(df["source"][3], 2.3, is_temp=True)

'Я с радостью помогу вам сгенерировать поле «О себе» для вашего резюме. У меня получилось придумать такой текст: «Я молодой и энергичный промоутер с четырьмя с половиной годами опыта работы. Я отлично разбираюсь в организации работы, раздаче листовок и расклейке листовок. Меня мотивирует возможность работать в команде, и я готов к новым вызовам. Я живу в Москве и имею водительское удостоверение. Я мотивирован на работу и готов принять вызов стать частью вашей команды».'

In [8]:
about_19 = []
about_23 = []
about_26 = []
about = []
for id, row in tqdm(df[:1000].iterrows()):
    about.append(predict(row["source"], 1.0, is_temp=False))
    about_19.append(predict(row["source"], 1.9, True))
    about_23.append(predict(row["source"], 2.3, True))
    about_26.append(predict(row["source"], 2.6, True))

1000it [3:18:44, 11.92s/it]


In [9]:
df_gen = pd.DataFrame(data={"full_sft_10epochs_no_temp": about,
                            "full_sft_10epochs_temp_19": about_19,
                            "full_sft_10epochs_temp_23": about_23,
                            "full_sft_10epochs_temp_26": about_26
                            })

In [10]:
df_gen.to_csv("full_sft_10epochs_from_0_to_999_with_temps.csv")